In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit

In [2]:
X, y, site = load_data()

2022-07-07 02:21:24.540002: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
from sklearn.utils import shuffle, resample

In [4]:
from sklearn.metrics import r2_score, median_absolute_error

In [5]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  "mlp4": {"model": mlp4, "lr": 0.001},
  # "cnn_vgg": {"model": cnn_vgg, "lr": 0.001},
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [6]:
n_runs = 10

In [7]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1502, 1334, 1576, 469, 1795])

In [8]:
def fit_and_eval(model, X, y, random_state, batch_size=128, n_epochs=1000, augment=None, train_size=None):
    model_func = model_dict[model]["model"]
    lr = model_dict[model]["lr"]
    X_train, X_test, y_train, y_test, site_train, site_test = train_test_split(X, y, site, test_size=0.2, random_state=random_state)
    imputer = SimpleImputer(strategy="median")
    # If train_size is set, select train_size subjects to be the training data:
    if train_size is not None:
        X_train, y_train, site_train = resample(X_train, y_train, site_train, replace=False, n_samples=train_size, random_state=random_state)
    
    # Impute train and test separately:
    X_train = np.concatenate([imputer.fit_transform(X_train[..., ii])[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([imputer.fit_transform(X_test[..., ii])[:, :, None] for ii in range(X_test.shape[-1])], -1)
    # Combat
    X_train = np.concatenate([CombatModel().fit_transform(X_train[..., ii], site_train[:, None], None, None)[:, :, None] for ii in range(X_train.shape[-1])], -1)
    X_test = np.concatenate([CombatModel().fit_transform(X_test[..., ii], site_test[:, None], None, None)[:, :, None] for ii in range(X_test.shape[-1])], -1)
    
    trained = model_fit(model_func, X_train, y_train, lr, 
                        batch_size=batch_size, n_epochs=n_epochs)
    metric = []
    value = []
    
    y_pred = trained.predict(X_test)
    metric.append("mae")
    value.append(mean_absolute_error(y_test, y_pred))
    metric.append("mad")
    value.append(median_absolute_error(y_test, y_pred))
    metric.append("r2")
    value.append(r2_score(y_test, y_pred))
    
    result = {'Model': [model] * len(metric),
              'Metric': metric,
              'Value': value}
    
    return pd.DataFrame(result), pd.DataFrame(dict(y_pred=y_pred.squeeze(), y_test=y_test))

In [9]:
dfs_eval = []
dfs_pred = []

In [10]:
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            X, 
            y, 
            random_state=seeds[ii],
            train_size=350)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("mlp4_4_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("mlp4_4_pred.csv")

##################################################
model:  mlp4
run:  0
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 72)]         0         
                                                                 
 flatten (Flatten)           (None, 7200)              0         
                                                                 
 dropout (Dropout)           (None, 7200)              0         
                                                                 
 dense (Dense)               (None, 500)               3600500   
                                                                 
 dropout_1 (Dropout)         (None, 500)               0         
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                       

2/3 [===================>..........] - ETA: 0s - loss: 11.2872 - mean_squared_error: 11.2872 - rmse: 3.3596 - mean_absolute_error: 2.7411
Epoch 00024: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
3/3 [==============================] - 0s 71ms/step - loss: 10.8305 - mean_squared_error: 10.8305 - rmse: 3.2910 - mean_absolute_error: 2.6956 - val_loss: 17.3710 - val_mean_squared_error: 17.3710 - val_rmse: 4.1679 - val_mean_absolute_error: 3.0412 - lr: 0.0010
Epoch 25/1000
3/3 [==============================] - 0s 69ms/step - loss: 11.3825 - mean_squared_error: 11.3825 - rmse: 3.3738 - mean_absolute_error: 2.6623 - val_loss: 15.5268 - val_mean_squared_error: 15.5268 - val_rmse: 3.9404 - val_mean_absolute_error: 2.8590 - lr: 5.0000e-04
Epoch 26/1000
3/3 [==============================] - 0s 55ms/step - loss: 11.5882 - mean_squared_error: 11.5882 - rmse: 3.4042 - mean_absolute_error: 2.6688 - val_loss: 13.0456 - val_mean_squared_error: 13.0456 - val_rmse: 3.6119 - val_me

Epoch 52/1000
3/3 [==============================] - 0s 54ms/step - loss: 10.6928 - mean_squared_error: 10.6928 - rmse: 3.2700 - mean_absolute_error: 2.5537 - val_loss: 15.4357 - val_mean_squared_error: 15.4357 - val_rmse: 3.9288 - val_mean_absolute_error: 2.8570 - lr: 2.5000e-04
Epoch 53/1000
3/3 [==============================] - 0s 68ms/step - loss: 11.3020 - mean_squared_error: 11.3020 - rmse: 3.3618 - mean_absolute_error: 2.6504 - val_loss: 15.5623 - val_mean_squared_error: 15.5623 - val_rmse: 3.9449 - val_mean_absolute_error: 2.8700 - lr: 2.5000e-04
Epoch 54/1000
3/3 [==============================] - 0s 65ms/step - loss: 10.9470 - mean_squared_error: 10.9470 - rmse: 3.3086 - mean_absolute_error: 2.5712 - val_loss: 14.7366 - val_mean_squared_error: 14.7366 - val_rmse: 3.8388 - val_mean_absolute_error: 2.7888 - lr: 2.5000e-04
Epoch 55/1000
3/3 [==============================] - 0s 47ms/step - loss: 10.3358 - mean_squared_error: 10.3358 - rmse: 3.2149 - mean_absolute_error: 2.5175 

3/3 [==============================] - 0s 74ms/step - loss: 10.8295 - mean_squared_error: 10.8295 - rmse: 3.2908 - mean_absolute_error: 2.6284 - val_loss: 14.9397 - val_mean_squared_error: 14.9397 - val_rmse: 3.8652 - val_mean_absolute_error: 2.8109 - lr: 1.2500e-04
Epoch 81/1000
3/3 [==============================] - 0s 63ms/step - loss: 10.3499 - mean_squared_error: 10.3499 - rmse: 3.2171 - mean_absolute_error: 2.5736 - val_loss: 15.3622 - val_mean_squared_error: 15.3622 - val_rmse: 3.9195 - val_mean_absolute_error: 2.8524 - lr: 1.2500e-04
Epoch 82/1000
3/3 [==============================] - 0s 71ms/step - loss: 10.2455 - mean_squared_error: 10.2455 - rmse: 3.2009 - mean_absolute_error: 2.5661 - val_loss: 16.5164 - val_mean_squared_error: 16.5164 - val_rmse: 4.0640 - val_mean_absolute_error: 2.9629 - lr: 1.2500e-04
Epoch 83/1000
3/3 [==============================] - 0s 77ms/step - loss: 10.3971 - mean_squared_error: 10.3971 - rmse: 3.2245 - mean_absolute_error: 2.5624 - val_loss: 17

3/3 [==============================] - 1s 149ms/step - loss: 157.3643 - mean_squared_error: 157.3643 - rmse: 12.5445 - mean_absolute_error: 11.5622 - val_loss: 54.1240 - val_mean_squared_error: 54.1240 - val_rmse: 7.3569 - val_mean_absolute_error: 6.4402 - lr: 0.0010
Epoch 2/1000
3/3 [==============================] - 0s 68ms/step - loss: 29.0762 - mean_squared_error: 29.0762 - rmse: 5.3922 - mean_absolute_error: 4.5188 - val_loss: 12.0163 - val_mean_squared_error: 12.0163 - val_rmse: 3.4665 - val_mean_absolute_error: 3.0642 - lr: 0.0010
Epoch 3/1000
3/3 [==============================] - 0s 41ms/step - loss: 14.5110 - mean_squared_error: 14.5110 - rmse: 3.8093 - mean_absolute_error: 3.0694 - val_loss: 31.7380 - val_mean_squared_error: 31.7380 - val_rmse: 5.6336 - val_mean_absolute_error: 4.4915 - lr: 0.0010
Epoch 4/1000
3/3 [==============================] - 0s 76ms/step - loss: 20.3065 - mean_squared_error: 20.3065 - rmse: 4.5063 - mean_absolute_error: 3.5538 - val_loss: 11.1559 - va

...

Epoch 90/1000
3/3 [==============================] - 0s 45ms/step - loss: 10.8480 - mean_squared_error: 10.8480 - rmse: 3.2936 - mean_absolute_error: 2.6016 - val_loss: 11.2131 - val_mean_squared_error: 11.2131 - val_rmse: 3.3486 - val_mean_absolute_error: 2.5202 - lr: 6.2500e-05
Epoch 91/1000
3/3 [==============================] - 0s 56ms/step - loss: 10.7035 - mean_squared_error: 10.7035 - rmse: 3.2716 - mean_absolute_error: 2.6380 - val_loss: 11.2055 - val_mean_squared_error: 11.2055 - val_rmse: 3.3475 - val_mean_absolute_error: 2.5189 - lr: 6.2500e-05
Epoch 92/1000
3/3 [==============================] - 0s 44ms/step - loss: 10.5917 - mean_squared_error: 10.5917 - rmse: 3.2545 - mean_absolute_error: 2.6327 - val_loss: 11.2952 - val_mean_squared_error: 11.2952 - val_rmse: 3.3608 - val_mean_absolute_error: 2.5270 - lr: 6.2500e-05
Epoch 93/1000
3/3 [==============================] - 0s 53ms/step - loss: 10.3804 - mean_squared_error: 10.3804 - rmse: 3.2219 - mean_absolute_error: 2.6002 

3/3 [==============================] - 0s 60ms/step - loss: 11.9833 - mean_squared_error: 11.9833 - rmse: 3.4617 - mean_absolute_error: 2.6313 - val_loss: 28.7300 - val_mean_squared_error: 28.7300 - val_rmse: 5.3600 - val_mean_absolute_error: 4.1458 - lr: 0.0010
Epoch 11/1000
3/3 [==============================] - 0s 58ms/step - loss: 14.5840 - mean_squared_error: 14.5840 - rmse: 3.8189 - mean_absolute_error: 2.9691 - val_loss: 18.4872 - val_mean_squared_error: 18.4872 - val_rmse: 4.2997 - val_mean_absolute_error: 3.3465 - lr: 0.0010
Epoch 12/1000
3/3 [==============================] - 0s 62ms/step - loss: 11.2365 - mean_squared_error: 11.2365 - rmse: 3.3521 - mean_absolute_error: 2.6508 - val_loss: 17.5774 - val_mean_squared_error: 17.5774 - val_rmse: 4.1925 - val_mean_absolute_error: 3.2828 - lr: 0.0010
Epoch 13/1000
3/3 [==============================] - 0s 54ms/step - loss: 12.5612 - mean_squared_error: 12.5612 - rmse: 3.5442 - mean_absolute_error: 2.8109 - val_loss: 28.3462 - val_

Epoch 39/1000
3/3 [==============================] - 0s 44ms/step - loss: 11.6173 - mean_squared_error: 11.6173 - rmse: 3.4084 - mean_absolute_error: 2.7188 - val_loss: 24.7118 - val_mean_squared_error: 24.7118 - val_rmse: 4.9711 - val_mean_absolute_error: 3.8068 - lr: 5.0000e-04
Epoch 40/1000
3/3 [==============================] - 0s 55ms/step - loss: 11.3444 - mean_squared_error: 11.3444 - rmse: 3.3681 - mean_absolute_error: 2.6053 - val_loss: 26.0298 - val_mean_squared_error: 26.0298 - val_rmse: 5.1019 - val_mean_absolute_error: 3.9154 - lr: 5.0000e-04
Epoch 41/1000
3/3 [==============================] - 0s 43ms/step - loss: 10.6130 - mean_squared_error: 10.6130 - rmse: 3.2578 - mean_absolute_error: 2.5522 - val_loss: 26.9914 - val_mean_squared_error: 26.9914 - val_rmse: 5.1953 - val_mean_absolute_error: 3.9972 - lr: 5.0000e-04
Epoch 42/1000
3/3 [==============================] - 0s 50ms/step - loss: 11.4129 - mean_squared_error: 11.4129 - rmse: 3.3783 - mean_absolute_error: 2.5912 

Epoch 67/1000
3/3 [==============================] - 0s 46ms/step - loss: 11.0516 - mean_squared_error: 11.0516 - rmse: 3.3244 - mean_absolute_error: 2.6289 - val_loss: 24.2283 - val_mean_squared_error: 24.2283 - val_rmse: 4.9222 - val_mean_absolute_error: 3.7665 - lr: 1.2500e-04
Epoch 68/1000
3/3 [==============================] - 0s 43ms/step - loss: 11.2233 - mean_squared_error: 11.2233 - rmse: 3.3501 - mean_absolute_error: 2.5987 - val_loss: 23.8484 - val_mean_squared_error: 23.8484 - val_rmse: 4.8835 - val_mean_absolute_error: 3.7349 - lr: 1.2500e-04
Epoch 69/1000
3/3 [==============================] - 0s 52ms/step - loss: 10.0454 - mean_squared_error: 10.0454 - rmse: 3.1695 - mean_absolute_error: 2.4590 - val_loss: 23.7635 - val_mean_squared_error: 23.7635 - val_rmse: 4.8748 - val_mean_absolute_error: 3.7276 - lr: 1.2500e-04
Epoch 70/1000
3/3 [==============================] - 0s 39ms/step - loss: 10.5283 - mean_squared_error: 10.5283 - rmse: 3.2447 - mean_absolute_error: 2.5851 

3/3 [==============================] - 0s 53ms/step - loss: 11.0304 - mean_squared_error: 11.0304 - rmse: 3.3212 - mean_absolute_error: 2.6181 - val_loss: 24.5577 - val_mean_squared_error: 24.5577 - val_rmse: 4.9556 - val_mean_absolute_error: 3.7957 - lr: 6.2500e-05
Epoch 96/1000
3/3 [==============================] - 0s 41ms/step - loss: 10.7395 - mean_squared_error: 10.7395 - rmse: 3.2771 - mean_absolute_error: 2.5354 - val_loss: 24.9821 - val_mean_squared_error: 24.9821 - val_rmse: 4.9982 - val_mean_absolute_error: 3.8314 - lr: 6.2500e-05
Epoch 97/1000
3/3 [==============================] - 0s 47ms/step - loss: 10.2851 - mean_squared_error: 10.2851 - rmse: 3.2070 - mean_absolute_error: 2.4974 - val_loss: 25.4932 - val_mean_squared_error: 25.4932 - val_rmse: 5.0491 - val_mean_absolute_error: 3.8747 - lr: 6.2500e-05
Epoch 98/1000
3/3 [==============================] - 0s 51ms/step - loss: 10.5876 - mean_squared_error: 10.5876 - rmse: 3.2539 - mean_absolute_error: 2.5334 - val_loss: 25

In [11]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)